# OpenStreetMap Project 

The objective of this OpenStreetMap (OSM) project is to audit and clean OSM data from the Greater Rochester, NY, area.  This region is of particular interest because it is where I was born and where I lived for the first eighteen years of my life. The raw data was downloaded from OSM via Mapzen and was saved as 'Rochester.osm' (see the Mapzen link [here](https://mapzen.com/data/metro-extracts/your-extracts/a299d201ae3b)). The size of the file was 53.6 MB. (A smaller sample of the Rochester.osm file, ~5 MB, is located in Rochester_sample.osm.) The data was processed and interrogated with a series of four scripts, summarized below:

* **audit_tags.py**: Summarize and categorize the data in 'tag' elements in the OSM file. Audit tags that contain (i) problematic characters, (ii) street addresses, (iii) cities, (iv) zip codes, and (v) phone numbers.
* **osm_to_csv.py**: Clean the OSM data and transfer to .csv files.
* **csv_to_database.py**: Define the schema for a SQL database and upload the data from the previously created .csv files.
* **sql_queries**: Extract information from the database with SQL queries.

Additionally, the schema.py script implemented in the Udacity lessons is included for the validation step in the osm_to_csv.py script. Resources that were utilized to develop the code in the python scripts are referenced in the code file headers.

---

## Section 1: Problems encountered in the map

### 1a: Cursory audit of 'tag' elements

To develop a rough sense of the data contained in the 'tag' elements of nodes, ways, and relations in the OSM file, the audit_tags.py script was run to summarize the contents of the 'tag' elements. In total, the OSM file contained 154,102 tag elements. In these elements, 446 _unique_ keys were present. After performing a cursory scan of the unique keys by eye, at least three large clusters were evident: (i) tags with 'tiger' data, (ii) tags with 'gnis' data, and (iii) tags labeled with 'fixme'. A fourth category was investigated, namely those tags that had keys with problematic characters as defined in one of the Udacity lessons (_Case study: OpenStreetMap data [SQL] / Quiz: Tag types_). All other keys were classified as 'other'. The tag elements were segmented into the five categories and counted programmatically, yielding the results in Table 1:

|Category |Number of keys          |Fraction of total|
|:--------|:----------------------:|----------------:|
|tiger    |61901                   |0.402            |
|gnis     |1873                    |0.012            |
|fixme    |36                      |0.000            |
|probem   |1                       |0.000            |
|other    |90291                   |0.586            |
|**TOTAL**|154102                  |1.000            |

<p style="text-align: center">
  <b>Table 1: Key categories from Rochester.osm.</b>
</p>

Forty percent of the tag data is from the United States Census Bureau's Topologically Integrated Geographic Encoding and Referencing (TIGER) system. According to OSM's wiki regarding [TIGER fixup] [1], a number of issues may be encountered with TIGER data. Since the TIGER database was created for the purpose of guiding census surveys, many of the issues deal with the accuracy of nodes representing roads and boundaries. Also, since the data was uploaded in 2007/2008, some of the data is antiquated. 

The next largest cluster of data, composing about 1% of the data, is from the United States Geographical Survey's Geographic Names Information System (GNIS). According to OSM's wiki regarding [USGS GNIS] [2], this data was also bulk imported like TIGER data, and hence contains a number of errors. Many of those errors relate to features that no longer exist. 

While issues with geographic location accuracy and outdated-ness are beyond the scope of this project, five problems were identified that could be addressed programmatically. They were: (1) keys with problematic characters, (2) overabbreviation of street names, (3) incorrect city values, (4) incorrect zip codes, and (5) non-uniform phone number formatting.

### 1b: Keys with problematic characters

To identify keys that contain problematic characters (characters other than alphanumeric and underscore), a regex was run against a dictionary of all the keys aggregated from the OSM file. Only one key was identified with problemmatic characters - 'Hours of Operation' - which contains spaces. During the upload of the OSM data to .csv files (with the osm_to_csv.py script), the spaces were replaced with underscores with a call to the following function. (Note: Docstrings are omitted below, but can be found in the script files.):

~~~~ python
def fix_prob_chars(key, problem_chars=PROBLEMCHARS):
    if problem_chars.search(key):
        new_key = re.sub(' ', '_', key)
    return new_key
~~~~

### 1c: Overabbrevation of street names

A larger issue was found with abbreviations in street names. First, a section of the auidit_tags.py script was run to compile all the tags with address-related fields. From that compilation, keys named 'addr:street' were identified as the most relevant. A second process was run to capture the last word at the end of street name strings, similar to the approach in the Udacity lesson _Case study: OpenStreetMap data [SQL] / Auditing Street Names_. After going through the collection of possible abbreviations manually, a mapping dictionary was developed to correlate abbrevations with their full form. During the upload of the OSM data to .csv files with osm_to_csv.py, the street names strings were interrogated for abbreviations, and the abbreviations were expanded:

~~~~ python
def fix_street_abbrevs(street):
    mapping = {
        'ave': 'Avenue',
        'Ave': 'Avenue',
        # ...
        # See code for complete mapping dict
    }
    
    elements = street.split()
    for i in range(len(elements)):
        if elements[i] in mapping:
            elements[i] = mapping[elements[i]]
    updated_street = ' '.join(elements)
    return updated_street
~~~~

### 1d: Incorrect city names

Several problems were identified with city names:
* Some values were misspelled or incorrect (_e.g._, 'East Rochester Town' should be just 'East Rochester').
* Certain values contained both the city and state.
* Some values contained a street address instead of the city.
* Certain values were not capitalized.  

To correct these issues, first all the words in city values were capitalized. Then, using data from the audit_tags.py script output, a mapping dictionary was implemented to change problematic values to corrected values:  
~~~~ python
def fix_cities(city):
    mapping = {
        'East Rochester Town': 'East Rochester',
        'Rochester, Ny': 'Rochester',
        'Rochestet': 'Rochester',
        'W Commercial St': 'East Rochester'
    }
    
    city = city.title()
    if city in mapping:
        city = mapping[city]
    return city
~~~~

### 1e: Incorrect zip codes

The OSM file was audited for correct zip codes. The audit_tags.py script compiled the values of tags with the key 'addr:postcode'. Only two instances were problematic - one with the value '1--', and a second with the value 'West Main Street'. During the conversion to .csv files, both of these zip codes were converted to 'fixme':

~~~~ python
def fix_zipcode(zipcode):
    zipformat = re.compile(r"(^[0-9]{5})(-[0-9]{4})?")
    if zipformat.match(zipcode):
        return zipcode
    else:
        return 'fixme'
~~~~

### 1f: Phone number formatting

Finally, all the values from tags with the key 'phone' were extracted from the OSM file and audited. Several different formatting schemes were present, so during the upload to csv files, all the formats were changed to a uniform layout. To accomplish this, first all number clusters were pulled from the phone number value. A restriction was placed on the size of the clusters, so that they could be between three to ten digits long, inclusive. Phone number values with no separating characters (such as spaces, hyphens, _etc._) resulted in one cluster of ten digits. Other values usually resulted in three clusters of 3 or 4 digits, but if the phone number included a three-digit country code, the value would yield four clusters. (Note that no extensions were present.) The country code was omitted, and then the remaining ten digits were formatted to a simple hyphenated layout: '123-456-7890':  
~~~~ python
def fix_phone_numbers(phone_number):
    # Find clusters of digits between 3 and 10 numbers in length
    number_re = re.compile(r'[0-9]{3,10}')
    digits = number_re.findall(phone_number)
    # If more than 3 clusters are present, then omit the first cluster, the 
    # country code (note: the initial audit showed that no extensions were 
    # present)
    if len(digits) > 3:
        digits = digits[1:]
    # Reformat the digits to '123-456-7890'
    digits = ''.join(digits)
    new_phone_number = '-'.join([digits[0:3], digits[3:6], digits[6:10]])
    return new_phone_number
~~~~  
---

## Section 2: Overview of the data

After loading the partially-cleaned data to a database with csv_to_database.py, a number of queries were performed with the sql_queries.py script. The subsections below give the query, followed by the result.  

### 2a: Total number of unique users
Borrowing a query from carlward's [sample_project.md] [3] file on github, the total number of contributors was extracted from the Rochester.osm file:  

Query:
~~~~ python
'''SELECT COUNT(DISTINCT(subquery.uid))
   FROM
       (SELECT uid FROM nodes
        UNION ALL
        SELECT uid FROM ways)
       AS subquery;'''
~~~~

Result: 451 unique users

### 2b: Top contributors  
The top ten contributors were identified:  

Query:
~~~~ python
'''SELECT subquery.user, count(*) AS num
   FROM
       (SELECT user from nodes
        UNION ALL
        SELECT user from ways)
       AS subquery
   GROUP BY subquery.user
   ORDER BY num DESC
   LIMIT 10;'''
~~~~  

Result:  

|User            |Number of contributions|
|:---------------|:---------------------:|
|woodpeck_fixbot |48647                  |
|sivart          |22160                  |
|timr            |18796                  |
|TomHynes        |18346                  |
|RussNelson      |14127                  |
|ECRock          |12809                  |
|dankpoet        |12238                  |
|URcommunications|10520                  |
|paperboat       |9358                   |
|stuuf           |8525                   |  

<p style="text-align: center">
  <b>Table 2: The top ten contributors to Rochester.osm.</b>
</p>

### 2c: Number of nodes
Query:  
~~~~ python
'''SELECT COUNT(*)
   FROM nodes;'''
~~~~  

Result: 236213 unique nodes

### 2d: Number of ways
Query:   
~~~~ python
'''SELECT COUNT(*)
   FROM ways;'''
~~~~  

Result: 30961 unique ways

### 2e: Number of restaurants
Query:  
~~~~ python
'''SELECT COUNT(DISTINCT(value))
   FROM 
       (SELECT value from nodes_tags
        UNION ALL
        SELECT value from ways_tags)
       AS subquery 
   WHERE subquery.value LIKE "%restaurant%";'''
~~~~  

Result: 25 restaurants

### 2f: Number of schools
Query:  
~~~~ python
'''SELECT COUNT(DISTINCT(value))
   FROM 
       (SELECT value from nodes_tags
        UNION ALL
        SELECT value from ways_tags)
       AS subquery 
   WHERE subquery.value LIKE "%school%";'''
~~~~  

Result: 271 shools  

### 2f: Number of nodes in ways
To investigate the number of nodes asssociated with ways, first the average was calculated:  

**Average**  
Query:  
~~~~ python
'''SELECT AVG(num)
   FROM
       (SELECT COUNT(node_id) AS num
        FROM ways_nodes
        GROUP BY id)
       AS subquery;'''
~~~~

Result: 9 nodes per way  

Then, to look at the distribution of nodes per way, another query was run:

**Distribution**  
Query:  
~~~~ python
'''SELECT COUNT(node_id) AS num
   FROM ways_nodes
   GROUP BY id
   ORDER BY num DESC;'''
~~~~

After, the data was displayed as a histogram in Figure 1 using matplotlib:  
![](figure_1.jpg)
<p style="text-align: center">
<b>Figure 1: Histogram of the distrubution of the number of nodes per way in Rochester.osm.</b>
</p>  

Immediately, one can see that the distribution is highly right-skewed. The mode is 5 nodes per way, but the average is 12 - leading to a hypothesis that most of the data represents simple man-made structures, and quadrilaterals are the simplest way of representing these structures. (5 nodes are needed to construct a quadrilateral. The first and last point coincide.) The entire lower end of the distribution shows a high preference for an odd number of nodes to be associated with a way, again suggesting a propensity for box-shaped artificial structures in the map data.  

**Ways with highest number of nodes**  
Ways that had a higher number of nodes per way tended to be associated with water features or governmental boundaries. The top two features, for instance, are a water hazard and a creek, respectively:  

Query:
~~~~ python
'''SELECT subquery.id, num, key, value, type
   FROM 
       (SELECT id, COUNT(node_id) AS num
        FROM ways_nodes
        GROUP BY id
        ORDER BY num DESC
        LIMIT 20) 
       AS subquery
   JOIN ways_tags
   ON subquery.id=ways_tags.id
   ORDER BY num DESC
   LIMIT 10;'''
~~~~

Result:  

|Way ID   |Number of nodes|Key        |Value                            |Type   |
|:--------|:-------------:|:----------|:--------------------------------|:------|
|231402426|560            |golf       |water_hazard                     |regular|
|231402426|560            |water      |stream                           |regular|
|145024913|534            |name       |Allen's Creek                    |regular|
|145024913|534            |layer      |-1                               |regular|
|145024913|534            |waterway   |stream                           |regular|
|37857171 |350            |source     |TIGER/Line® 2008 Place Shapefiles|regular|
|37857171 |350            |boundary   |administrative                   |regular|
|37857171 |350            |admin_level|8                                |regular|
|313476849|283            |source     |TIGER/Line® 2008 Place Shapefiles|regular|
|313476849|283            |boundary   |administrative                   |regular|  

<p style="text-align: center">
  <b>Table 3: The ways in Rochester.osm with the highest number of nodes.</b>
</p>

The top two features are displayed in the figures below:

![](figure_2.jpg)  
<p style="text-align: center">
<b>Figure 2: Render of way #231402426, a water hazard at a golf course (Credit: OpenStreetMap).</b>
</p> 

![](figure_3.jpg)  
<p style="text-align: center">
<b>Figure 3: Render of way #145024913, a stream in Penfield (Credit: OpenStreetMap).</b>
</p> 

---

## Section 3: Other ideas about the dataset

One of the largest issues with the data set was simply lack of data. For example, only 25 restaurants were tagged in the OSM file, whereas the US Census Bureau documented close to 600 restaurants in Rochester in 2012 ([link] [5]). One idea to increase the amount of data would be to set up an 'augmented' reality app that would identify structures that are not present in the OSM database, and offer incentives for scanning in their GPS data. One issue with this approach, however, could be the accuracy with the users' GPS hardware, which would likely be inferior to dedicated GPS systems. Also, incentivizing user activity could be problematic. Perhaps OSM data collection could be combined with some kind of exercise app with rewards for meeting certain movement goals.
 
 
[1]: http://wiki.openstreetmap.org/wiki/TIGER_fixup "http://wiki.openstreetmap.org/wiki/TIGER_fixup"
[2]: http://wiki.openstreetmap.org/wiki/USGS_GNIS "http://wiki.openstreetmap.org/wiki/USGS_GNIS"
[3]: https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md
[4]: https://www.linkedin.com/in/thomas-hynes-7ab13049/
[5]: https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=CF
 